In [ ]:
from flask import Flask, request, jsonify
import pickle
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import random
import numpy as np

app = Flask(__name__)

# 저장된 모델 및 인코더, 벡터라이저, TF-IDF 행렬 불러오기
with open('xgboost_model1.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

with open('label_encoder1.pkl', 'rb') as le_file:
    loaded_label_encoder = pickle.load(le_file)

with open('tfidf_vectorizer1.pkl', 'rb') as tv_file:
    loaded_tfidf_vectorizer = pickle.load(tv_file)

with open('tfidf_matrix1.pkl', 'rb') as tm_file:
    loaded_tfidf_matrix = pickle.load(tm_file)

# 데이터 불러오기
data = pd.read_csv('모델링DB용.csv')

# 필요한 열 제외
data = data.drop(columns=['curArtist', 'curTitle'])

# curExplain, curHead, curObject를 결합하여 새로운 컬럼 생성
data['combined'] = data['curExplain'].astype(str) + ' ' + data['curHead'].astype(str) + ' ' + data['curObject'].astype(str)

# 범주형 변수를 category 타입으로 변환
categorical_cols = ['curColor', 'curColor2', 'curColor3', 'genre', 'song']
for col in categorical_cols:
    data[col] = data[col].astype('category')

# 'song' 열을 Label Encoding
data['song'] = loaded_label_encoder.transform(data['song'])

# 예측 수행 및 랜덤 추천 함수
def recommend_random_songs(model, label_encoder, tfidf_vectorizer, tfidf_matrix, data, input_keywords, input_genre, n_recommendations=500, n_random=20):
    # 입력 키워드를 TF-IDF 벡터화
    input_vector = tfidf_vectorizer.transform([input_keywords])
    
    # 코사인 유사도 계산
    cosine_sim = cosine_similarity(input_vector, tfidf_matrix)
    
    # 입력한 장르와 일치하는 인덱스 필터링
    genre_indices = data[data['genre'] == input_genre].index
    
    # 유사도가 높은 순서로 정렬된 인덱스
    sorted_indices = genre_indices[np.argsort(-cosine_sim[0][genre_indices])]
    
    # 상위 n개의 유사한 노래 추천
    recommended_song_indices = sorted_indices[:n_recommendations]
    recommended_songs = label_encoder.inverse_transform(data.loc[recommended_song_indices, 'song'])
    
    # n개의 랜덤 추천 노래 선택
    random_recommendations = random.sample(list(recommended_songs), n_random)
    
    return random_recommendations

# 추천 API 엔드포인트 정의
@app.route('/recommend', methods=['POST'])
def recommend():
    content = request.json
    input_keywords = content['keywords']
    input_genre = content['genre']
    
    recommendations = recommend_random_songs(loaded_model, loaded_label_encoder, loaded_tfidf_vectorizer, loaded_tfidf_matrix, data, input_keywords, input_genre)
    
    return jsonify(recommendations)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.219.65:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Jun/2024 08:34:42] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2024 08:34:53] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2024 08:34:53] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2024 08:34:53] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2024 08:34:53] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2024 08:34:53] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2024 08:34:53] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2024 08:34:53] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2024 08:34:53] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2024 08:34:53] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2024 08:35:58] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2024 08:36:26] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/20